In [ ]:
!pip install stable-baselines3[extra]

## The old code

In [ ]:
import random

import pandas as pd
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import gc
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv
from gym.spaces import Discrete, MultiDiscrete, Box
from stable_baselines3 import PPO
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
from pandas import Series
from numpy.random import randn


myglobalList = []
class cache_env(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, dataset_df):
        super(cache_env, self).__init__()
        """   """
        self.data = dataset_df
        self.current_step = 0
        self.done = False
        #self.MAX_STEPS = 50000
        self.fresh_cost_weight = 1
        self.reward = 0
        self.episodes = []
        self.current_time = 0
        self.num_operations = len(self.data["Id"].unique())
    
        self.cache_hit = len(self.data["Request"])
        self.max_steps = 99
        self.alpha = 0.3
        self.gamma = 0.9
        self.beta = 0.5
        self.wrong_action = 0
        self.req =[ 0 , 1 , 2  ,3 , 4 , 5 , 6  ,7 , 8 , 9, 10 ,11, 12, 13, 14, 15,16 ,17 ,18 ,19 ,20 ,21, 22 ,23, 24 ,25 ,26, 27 ,28, 29, 30, 31, 32, 33, 34, 35 ,36 ,37 ,38, 39 ,40, 41, 42, 43, 44, 45, 46, 47, 48 ,49, 50, 51, 52, 53, 54 ,55, 56, 57, 58, 59 ,60 ,61 ,62 ,63 ,64 ,65, 66 ,67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80 ,81, 82, 83, 84 ,85, 86 ,87 ,88 ,89, 90, 91, 92, 93, 94, 95, 96, 97, 98 ,99]
        self.Freshness = 2
        self.p= [0.010650295689525247, 0.005043648733142891, 0.022178817410027585, 0.004530316901020681, 0.0006137003016316589, 0.018686778333924625, 0.0524937555696211, 0.006439772278007516, 0.012706965345992582, 0.016580744410421233, 0.004795111427467691, 0.015235033865786285, 0.01701074037975514, 0.003568042004335914, 0.015610415901294417, 0.0034975710751972144, 0.005609438917355658, 0.012361625212080063, 0.016805314049332498, 0.0035146050872414403, 0.0021889911104261417, 0.00013723476596461115, 0.01642291061524337, 0.00037923769039249766, 0.0030852415221387375, 0.0028784519536756425, 0.0027917633833648425, 0.0006065679679477798, 0.008782138378110482, 0.006642782737233288, 0.017883085822997247, 0.0011088736756136308, 0.015785356871808265, 0.009716256613829022, 0.005369858080857437, 0.005992321838491632, 0.0012311507006249611, 0.04497207816533788, 0.0006279884385921308, 0.012441793329102002, 0.04525276621456287, 0.0030388434870523007, 0.021133282873615933, 0.0019690033197928836, 0.01171841064628898, 0.008991985496192845, 0.0037661023729628207, 0.006934135227430138, 0.02642419337482771, 0.001791265150008825, 0.0069295363754308115, 0.013256958571097321, 0.00816787869525204, 0.00029571928442151396, 0.010140634322939213, 0.01779432488267807, 0.030393248337734824, 0.005147075966076997, 0.005094099211606196, 0.028768540174639368, 0.006752988147868955, 0.013105068191435754, 0.005438973125311497, 0.011287871057733656, 0.0001714536553515389, 0.009549244005053303, 0.007133419013666931, 0.0023754341644621757, 0.019677040952414404, 0.0007211457930846876, 0.001472031488805814, 0.0023882202804038632, 0.009243977427379702, 0.006923496709328014, 0.022980458022592187, 0.010036298779444786, 0.0012478254962180468, 0.003997547510156685, 0.01933221273731145, 0.022809655018506847, 0.009761126337217707, 0.001589854812818814, 0.0025293576173326393, 0.008665642013404362, 0.007750203793041921, 0.010481041454692086, 0.001327149625972468, 0.0011747868024911678, 0.0040832724083620375, 0.0028973421676875555, 0.034291515616839296, 0.00016254066563481384, 0.024861836178396657, 0.016020316973436866, 0.0016355758960824307, 0.012386405149080182, 0.0027327358735940166, 0.004827396229274639, 0.011556991407477457, 0.0006357668611078893]
        self.clock = []
        self.data.to_dict('index')
       
        
        self.num_nodes = 3
        self.my_nodes = {}
        for i in range(self.num_nodes):
            self.my_nodes[i] = self.data[i*25:i*25+25]
        self.my_nodes[3] = self.data

        self.nodeMap =  [[0, 1, 2, 3], [1, 0, 2, 3], [2, 1, 0, 3], [3, 0,1 ,2 ]]
        self.LATENCYMATRIX = [[20, 10, 5, 0], [10, 20,10, 0], [5, 10, 20, 0], [0,0,0,20]]
        self.TRAFFICMATRIX  = [[3, 2, 1, 0], [2, 3, 2, 0], [1, 2, 3, 0], [0, 0, 0,3]]
        self.CARBONMATRIX =  [[3, 2, 1, 0], [2, 3, 2, 0], [1, 2, 3, 0], [0, 0, 0,3]]




        action = [[0, 1, 2], [0, 1]]
        self.result = list(product(*action))

        self.action_space = spaces.Discrete(len(self.result))
    

        self.observation_space = spaces.MultiDiscrete([self.num_nodes, self.num_operations, self.Freshness])  # self.cache_sehit] )

    def searchFunction(self, a1):
        print(a1)

        for i in range(4):
          if self.request in self.my_nodes[self.nodeMap[a1][i]]["Id"].values:
            return  self.nodeMap[a1][i]
          
        return 3
 

        return found

    def get_observation(self):
      
        self.num_nodes = np.random.randint(0, high=3, size=1, dtype=int)
        self.num_nodes = self.num_nodes[0]
        new_p = self.p[self.num_nodes:] + self.p[:self.num_nodes]
        self.request = np.random.choice(self.req ,1, p= new_p)
        self.request = self.request[0]
        self.Freshness = np.random.randint(2)      
        return np.array([self.num_nodes, self.request, self.Freshness])


    def step(self, action):

        a1 = self.result[action][0]

        a2 = self.result[action][1]

        if self.current_step == 0:
            self.episodes = []
        self.reward = 0
        
        self.current_time = (self.current_time + random.randint(5,10)) % 100

 
        self.request =  np.random.choice(self.req , 1, self.p)
        self.request = np.array(self.request)
        self.request = self.request[0]
        self.Tage = self.current_time - (self.my_nodes[self.searchFunction(a1)].loc[self.my_nodes[self.searchFunction(a1)]['Id'] == self.request, 'Timestamp'].values)
        self.Tage = self.Tage[0]
        self.lifetime = self.my_nodes[self.searchFunction(a1)].loc[self.my_nodes[self.searchFunction(a1)]['Id'] == self.request, 'Lifetime'].values
   
        self.penalization1 = self.LATENCYMATRIX[a1][self.searchFunction(a1)]
        self.penalization2 = self.TRAFFICMATRIX[a1][self.searchFunction(a1)]
        self.penalization3 = self.CARBONMATRIX[a1][self.searchFunction(a1)]
    
        if a1 == 0 and a2 == 0:

            found_in = self.searchFunction(a1)
            
            if (self.Tage < self.lifetime).all() :

              self.Freshness = 1
              self.wrong_action = 0 

            else:

                self.Freshness = 0
                self.wrong_action = 1
            
            self.reward  += self.beta*(self.wrong_action)
        elif a1 == 0 and a2 == 1:
            found_in = self.searchFunction(a1)


            if (self.Tage < self.lifetime).all() :
   
                self.freshness = 1
                self.wrong_space = 1

            else:

                self.Freshness = 0
                self.wrong_action = 0
            self.reward += self.gamma*(self.penalization1/25)+ self.alpha*(self.penalization2/3) + self.beta*(self.penalization3) + self.wrong_action 
 
            data_f = self.my_nodes[found_in].loc[self.my_nodes[found_in]['Id'] == self.request]
            data_f.iloc[-1, data_f.columns.get_loc('freshness')] = self.Tage
  
            new_row = {"Id": data_f.iat[0, 0], "Request": data_f.iat[0, 1], "Lifetime": data_f.iat[0, 2], "Timestamp": data_f.iat[0, 3], "Prob": data_f.iat[0, 4]}

            self.my_nodes[a1] = self.my_nodes[a1].append(new_row, ignore_index=True)
            self.my_nodes[a1].drop(self.my_nodes[a1]['freshness'].idxmax())



        elif a1 == 1 and a2 == 0:

            found_in = self.searchFunction(a1)

            if (self.Tage < self.lifetime).all() :
 
              self.Freshness = 1
              self.wrong_action = 0 

            else:

                self.Freshness = 0
                self.wrong_action = 1
            self.reward  += self.beta*(self.wrong_action)

        elif a1 == 1 and a2 == 1:
          
            found_in = self.searchFunction(a1)
            #print("the data is found in " + str(found_in))
            if (self.Tage < self.lifetime).all() :
                #print("The data is fresh")
                self.Freshness = 1
                self.wrong_action = 1

            else:
                #print("The data is expired")
                self.Freshness = 0
                self.wrong_action = 0
            self.reward += self.gamma*(self.penalization1/25 )+ self.alpha*(self.penalization2/3) + self.beta*(self.penalization3) + self.wrong_action 
            #self.reward -= self.gamma*(self.penalization1/25 )
            data_f = self.my_nodes[found_in].loc[self.my_nodes[found_in]['Id'] == self.request]
            data_f.iloc[-1, data_f.columns.get_loc('freshness')] = self.Tage
            new_row = {"Id": data_f.iat[0, 0], "Request": data_f.iat[0, 1], "Lifetime": data_f.iat[0, 2], "Timestamp": data_f.iat[0, 3], "Prob": data_f.iat[0, 4]}
            #print(new_row)
            self.my_nodes[a1] = self.my_nodes[a1].append(new_row, ignore_index=True)
            self.my_nodes[a1] = self.my_nodes[a1]
            self.my_nodes[a1].drop(self.my_nodes[a1]['freshness'].idxmax())

            
            

        elif a1 == 2 and a2 == 0:
            
            found_in = self.searchFunction(a1)
            #print("the data is found in " + str(found_in))

            if (self.Tage < self.lifetime).all() :
             # print("The data is fresh")
              self.Freshness = 1
              self.wrong_action = 0 

            else:
                #print("The data is expired")
                self.Freshness = 0
                self.wrong_action = 1
            self.reward  += self.beta*(self.wrong_action)

        elif a1 == 2 and a2 == 1:
         
            found_in = self.searchFunction(a1)
           # print("the data is found in " + str(found_in))
            if (self.Tage < self.lifetime).all() :
                #print("The data is fresh")
                self.Freshness = 1
                self.wrong_action = 1

            else:
                #print("The data is expired")
                self.Freshness = 0
                self.wrong_action = 0
            self.reward += self.gamma*(self.penalization1/25 )+ self.alpha*(self.penalization2/3) + self.beta*(self.penalization3) + self.wrong_action 
            #self.reward -= self.gamma*(self.penalization1/25 )
            data_f = self.my_nodes[found_in].loc[self.my_nodes[found_in]['Id'] == self.request]
            data_f.iloc[-1, data_f.columns.get_loc('freshness')] = self.Tage
            new_row = {"Id": data_f.iat[0, 0], "Request": data_f.iat[0, 1], "Lifetime": data_f.iat[0, 2], "Timestamp": data_f.iat[0, 3], "Prob": data_f.iat[0, 4]}
            #print(new_row)
            self.my_nodes[a1] = self.my_nodes[a1].append(new_row, ignore_index=True)
            self.my_nodes[a1].drop(self.my_nodes[a1]['freshness'].idxmax())

        #myglobalList.append(self.beta*(self.penalization3))
        #print(myglobalList )
        self.current_step += 1
        #self.request += 1|
        #self.episodes.append(self.reward)
        done = self.current_step> self.max_steps 
        return self.get_observation(), self.reward, done, {}

    def reset(self):
        self.request = 3
        self.num_operations = len(self.data["Id"].unique())
        self.Freshness = 2
        self.current_step = 0
        self.current_time = 0

        # obs = np.array [self.my_nodes, self.num_operations,self.Freshness]
        # return observatio
        return self.get_observation()

    def render(self, mode='human', close=False):
        """   """
        print(f'Step: {self.current_step}')
        # print(f'Next Requests: {self.data.loc[self.current_step.values[0]]}')
        print(f'Next Requests: {np.random.choice(self.req , 1, p= [0.010650295689525247, 0.005043648733142891, 0.022178817410027585, 0.004530316901020681, 0.0006137003016316589, 0.018686778333924625, 0.0524937555696211, 0.006439772278007516, 0.012706965345992582, 0.016580744410421233, 0.004795111427467691, 0.015235033865786285, 0.01701074037975514, 0.003568042004335914, 0.015610415901294417, 0.0034975710751972144, 0.005609438917355658, 0.012361625212080063, 0.016805314049332498, 0.0035146050872414403, 0.0021889911104261417, 0.00013723476596461115, 0.01642291061524337, 0.00037923769039249766, 0.0030852415221387375, 0.0028784519536756425, 0.0027917633833648425, 0.0006065679679477798, 0.008782138378110482, 0.006642782737233288, 0.017883085822997247, 0.0011088736756136308, 0.015785356871808265, 0.009716256613829022, 0.005369858080857437, 0.005992321838491632, 0.0012311507006249611, 0.04497207816533788, 0.0006279884385921308, 0.012441793329102002, 0.04525276621456287, 0.0030388434870523007, 0.021133282873615933, 0.0019690033197928836, 0.01171841064628898, 0.008991985496192845, 0.0037661023729628207, 0.006934135227430138, 0.02642419337482771, 0.001791265150008825, 0.0069295363754308115, 0.013256958571097321, 0.00816787869525204, 0.00029571928442151396, 0.010140634322939213, 0.01779432488267807, 0.030393248337734824, 0.005147075966076997, 0.005094099211606196, 0.028768540174639368, 0.006752988147868955, 0.013105068191435754, 0.005438973125311497, 0.011287871057733656, 0.0001714536553515389, 0.009549244005053303, 0.007133419013666931, 0.0023754341644621757, 0.019677040952414404, 0.0007211457930846876, 0.001472031488805814, 0.0023882202804038632, 0.009243977427379702, 0.006923496709328014, 0.022980458022592187, 0.010036298779444786, 0.0012478254962180468, 0.003997547510156685, 0.01933221273731145, 0.022809655018506847, 0.009761126337217707, 0.001589854812818814, 0.0025293576173326393, 0.008665642013404362, 0.007750203793041921, 0.010481041454692086, 0.001327149625972468, 0.0011747868024911678, 0.0040832724083620375, 0.0028973421676875555, 0.034291515616839296, 0.00016254066563481384, 0.024861836178396657, 0.016020316973436866, 0.0016355758960824307, 0.012386405149080182, 0.0027327358735940166, 0.004827396229274639, 0.011556991407477457, 0.0006357668611078893])}')
        print(f'Reward: {self.reward}')
        #print(f'Freshness: {self.Freshness}')
        #print(f'Latenc cost: {self.penalization1}')
        #print(f'Traffic cost: {self.penalization2}')
        if self.current_step == 500:
            plt.plot(self.episodes)
            
        print(len(self.episodes))

In [ ]:
from stable_baselines3.common.env_checker import check_env 
df = pd.read_csv('dataset.csv')
env = cache_env(df)
check_env(env)